In [1]:
import pandas as pd
import datetime
import pytz
import numpy as np
charging_session_data=pd.read_pickle('data/charging_session_data_sample.pkl')
start_date=datetime.datetime(2022,1,1,tzinfo=pytz.timezone('CET'))
end_date=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))
timesteplist=pd.date_range(start=start_date,end=end_date+datetime.timedelta(days=2),freq='15Min',tz='CET')
day_ahead_prices=pd.read_pickle('data/day_ahead_market_prices_NL.pkl')

In [2]:
from helperfunctions.uncontrolled_charging_model import uncontrolled_charging

uncontrolled_charging_total=uncontrolled_charging(charging_session_data,timesteplist)
print('Uncontrolled charging profiles for all charging stations modelled')

Uncontrolled charging profiles for all charging stations modelled


In [3]:
from helperfunctions.volumetric_ToU_model import volumetric_ToU

grid_tariff_df=pd.DataFrame(0,index=timesteplist,columns=['Grid tariff (€/kWh)'])
DA_charging_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
for CS in charging_session_data['Charging station ID'].unique():
    charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
    DA_charging_total[CS]=volumetric_ToU(charging_session_data_CS,timesteplist,CS,grid_tariff_df,day_ahead_prices,True)
    print('Optimal dynamic retail price charging profile for '+CS+' modelled')

Set parameter Username
Set parameter LicenseID to value 2656772
Academic license - for non-commercial use only - expires 2026-04-25
Optimal dynamic retail price charging profile for Charging station 1 modelled
Optimal dynamic retail price charging profile for Charging station 2 modelled


In [4]:
for low_tariff_value in [0.01,0.02,0.03]:
    medium_tariff_value=low_tariff_value*2
    high_tariff_value=low_tariff_value*3

    grid_tariff_df=pd.DataFrame(0,index=timesteplist,columns=['Grid tariff (€/kWh)'])
    high_tariff_hours=[18,19,20,21]
    medium_tariff_hours=[22,23,0,16,17]
    low_tariff_hours=list(range(1,16))
    grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(high_tariff_hours),high_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
    grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(medium_tariff_hours),medium_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
    grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(low_tariff_hours),low_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
    for dynamic_retail_prices_considered in [True, False]:
        volumetric_ToU_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
        for CS in charging_session_data['Charging station ID'].unique():
            charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
            volumetric_ToU_total[CS]=volumetric_ToU(charging_session_data_CS,timesteplist,CS,grid_tariff_df,day_ahead_prices,dynamic_retail_prices_considered)
        if dynamic_retail_prices_considered==True:
            print('Volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for dynamic retail prices')
        else:
            print('Volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for fixed retail prices')

Volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for dynamic retail prices
Volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for fixed retail prices
Volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for dynamic retail prices
Volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for fixed retail prices
Volumetric ToU profiles modelled for low tariff value of 0.03 €/kWh and for dynamic retail prices
Volumetric ToU profiles modelled for low tariff value of 0.03 €/kWh and for fixed retail prices


In [5]:
from helperfunctions.segmented_volumetric_ToU_model import segmented_volumetric_ToU


for low_tariff_value in [0.01,0.02,0.03]:
    medium_tariff_value=low_tariff_value*2
    high_tariff_value=low_tariff_value*3

    grid_tariff_df=pd.DataFrame(0,index=timesteplist,columns=['Grid tariff (€/kWh)'])
    high_tariff_hours=[18,19,20,21]
    medium_tariff_hours=[22,23,0,16,17]
    low_tariff_hours=list(range(1,16))
    tariff_threshold_df=pd.DataFrame(index=timesteplist,columns=['Threshold_1','Threshold_2'])
    tariff_threshold_df['Threshold_1']=np.where(tariff_threshold_df.index.hour.isin(low_tariff_hours),4,0)
    tariff_threshold_df['Threshold_2']=np.where(tariff_threshold_df.index.hour.isin(high_tariff_hours),0,4)  
    for dynamic_retail_prices_considered in [True, False]:
        segmented_volumetric_ToU_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
        for CS in charging_session_data['Charging station ID'].unique():
            charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
            segmented_volumetric_ToU_total[CS]=segmented_volumetric_ToU(charging_session_data_CS,timesteplist,CS,tariff_threshold_df,day_ahead_prices,dynamic_retail_prices_considered,low_tariff_value,medium_tariff_value,high_tariff_value)
        if dynamic_retail_prices_considered==True:
            print('Segmented volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for dynamic retail prices')
        else:
            print('Segmented volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for fixed retail prices')

Segmented volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for dynamic retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for fixed retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for dynamic retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for fixed retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.03 €/kWh and for dynamic retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.03 €/kWh and for fixed retail prices
